<a href="https://colab.research.google.com/github/CarineTalandier/deepLearning/blob/main/projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [181]:
# IMPORT
import numpy as np
import os
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.optimizers import *
from keras import regularizers
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.optimizers import Adamax

  

In [182]:
def launchConversion(pathData, pathNumpy, resizeImg, imgSize):
    """
    # Permet de lancer la conversion des images en tableau numpy
    :param pathData: chemin ou sont les
    :param pathNumpy:
    :param resizeImg:
    :param imgSize:
    """

    #Pour chaque classe
    for mealClasse in os.listdir(pathData):
        pathMeal = pathData + '/' + mealClasse
        imgs = []

        #Pour chaque image d'une classe, on la charge, resize et transforme en tableau
        for imgMeal in tqdm(os.listdir(pathMeal), "Conversion de la classe : '{}'".format(mealClasse)):
            imgMealPath = pathMeal + '/' + imgMeal
            img = Image.open(imgMealPath)
            img.load()
            if resizeImg == True:
                img = img.resize(size=imgSize)

            data = np.asarray(img, dtype=np.float32)
            imgs.append(data)

        #Converti les gradients de pixels (allant de 0 à 255) vers des gradients compris entre 0 et 1
        imgs = np.asarray(imgs) / 255.

        #Enregistre une classe entiere en un fichier numpy
        np.save(pathNumpy + '/' + mealClasse + '.npy', imgs)

In [183]:
pathNumpy = '/content/numpy'
pathData = '/content/Dataset'
resizeImg = True
imgSize = (50, 50)
launchConversion(pathData, pathNumpy, resizeImg, imgSize)

Conversion de la classe : '.ipynb_checkpoints': 0it [00:00, ?it/s]
Conversion de la classe : 'beet_salad': 100%|██████████| 6/6 [00:00<00:00, 175.98it/s]


In [184]:
def get_labels(path):
    """
    # Permet de recuperer les labels de nos classe, leurs indices dans le tableau et leur matrix binaire one hot encoder
    :param path: chemin ou sont stocké nos fichiers Numpy
    """

    labels = [file.replace('.npy', '') for file in os.listdir(path) if file.endswith('.npy')]
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [185]:
def get_train_test(train_ratio, pathData):
    """
    # Retourner le dataset melanger en dataset d'entrainement et de validation selon un ratio
    :param train_ratio: permet de gerer la part entre dataset de train et de validation
    :param pathData: chemin des fichiers numpy
    """

    labels, _, _ = get_labels(pathData)
    classNumber = 0
 
    #On init avec le premier tableau pour avoir les bonnes dimensions pour la suite
    X = data = np.load(pathData + '/' + labels[0] + '.npy')
    Y = np.zeros(X.shape[0])
    dimension = X[0].shape
    classNumber += 1  

    #On ajoute le reste des fichiers numpy de nos classes
    for i, label in enumerate(labels[2:]):
      data = np.load(pathData + '/' + label + '.npy')
      X = np.vstack((X, data))
      Y = np.append(Y, np.full(data.shape[0], fill_value=(i+1)))
      classNumber += 1
  

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=train_ratio)
    return X_train, X_test, to_categorical(Y_train), to_categorical(Y_test), classNumber, dimension

In [191]:
#Definition des chemins et autres variables
pathData = '/content/numpy'
trainRatio = 0.8
epochs = 1000
batch_size = 16
earlyStopPatience = 5

 

#Permet de stopper l'entrainement quand le modèle n'entraine pluss
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=earlyStopPatience, verbose=0, mode='auto')

#Permet de sauvegarder le model a chaque iteration si il est meilleur que le precedent
check = ModelCheckpoint('/content/trainedModel/moModel.hdf5', monitor='val_loss', verbose=0,
                        save_best_only=True, save_weights_only=False, mode='auto')

#Recuperation de nos data pré traité
x_train, x_test, y_train, y_test, classNumber, dimension = get_train_test(trainRatio, pathData)

#On verifie les dimensions de nos données
print('DIMENSION X TRAIN ' + str(x_train.shape))
print('DIMENSION X TEST ' + str(x_test.shape))
print('DIMENSION Y TRAIN ' + str(y_train.shape))
print('DIMENSION Y TEST ' + str(y_test.shape))


#On creer le modele
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(dimension[0], dimension[1], dimension[2])))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(classNumber, activation='softmax'))

#On compile le modele
model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0),
                  metrics=['accuracy'])

#On lance l'entrainement du modele
trainning = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))



DIMENSION X TRAIN (67, 50, 50, 3)
DIMENSION X TEST (17, 50, 50, 3)
DIMENSION Y TRAIN (67, 4)
DIMENSION Y TEST (17, 4)
Epoch 1/1000


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adamax.py:90: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adamax, self).__init__(name, **kwargs)


5/5 [==============================] - 1s 87ms/step - loss: 1.0208 - accuracy: 0.7612 - val_loss: 1.5001 - val_accuracy: 0.7059
Epoch 2/1000
5/5 [==============================] - 0s 48ms/step - loss: 0.8469 - accuracy: 0.8060 - val_loss: 1.0208 - val_accuracy: 0.7059
Epoch 3/1000
5/5 [==============================] - 0s 46ms/step - loss: 0.6989 - accuracy: 0.8060 - val_loss: 1.1535 - val_accuracy: 0.7059
Epoch 4/1000
5/5 [==============================] - 0s 45ms/step - loss: 0.6664 - accuracy: 0.8060 - val_loss: 1.2919 - val_accuracy: 0.7059
Epoch 5/1000
5/5 [==============================] - 0s 48ms/step - loss: 0.5683 - accuracy: 0.8060 - val_loss: 1.1471 - val_accuracy: 0.7059
Epoch 6/1000
5/5 [==============================] - 0s 49ms/step - loss: 0.5284 - accuracy: 0.8209 - val_loss: 1.0721 - val_accuracy: 0.7059
Epoch 7/1000
5/5 [==============================] - 0s 49ms/step - loss: 0.4980 - accuracy: 0.8209 - val_loss: 1.0831 - val_accuracy: 0.7059
Epoch 8/1000
5/5 [========

TRAIN

PREDICTION

In [ ]:
import time
from keras.models import load_model

def predict(modelPath,imagePath, imageSize, label):
    """
    # Fonction qui permet de convertir une image en array, de charger le modele et de lui injecter notre image pour une prediction
    :param modelPath: chemin du modèle au format hdf5
    :param imagePath: chemin de l'image pour realiser une prediction
    :param imageSize: défini la taille de l'image. IMPORTANT : doit être de la même taille que celle des images
    du dataset d'entrainements
    :param label: nom de nos 5 classes de sortie
    """

    start = time.time()

    # Chargement du modele
    print("Chargement du modèle :\n")
    model = load_model(modelPath)
    print("\nModel chargé.")

    #Chargement de notre image et traitement
    data = []
    img = Image.open(imagePath)
    img.load()
    img = img.resize(size=imageSize)
    img = np.asarray(img) / 255.
    data.append(img)
    data = np.asarray(data)

    #On reshape pour correspondre aux dimensions de notre modele
    # Arg1 : correspond au nombre d'image que on injecte
    # Arg2 : correspond a la largeur de l'image
    # Arg3 : correspond a la hauteur de l'image
    # Arg4 : correspond au nombre de canaux de l'image (1 grayscale, 3 couleurs)
    dimension = data[0].shape

    #Reshape pour passer de 3 à 4 dimension pour notre réseau
    data = data.astype(np.float32).reshape(data.shape[0], dimension[0], dimension[1], dimension[2])

    #On realise une prediction
    prediction = model.predict(data)


    #On recupere le numero de label qui a la plus haut prediction
    maxPredict = np.argmax(prediction)

    #On recupere le mot correspondant à l'indice precedent
    word = label[maxPredict]
    pred = prediction[0][maxPredict] * 100.
    end = time.time()


    #On affiche les prédictions
    print()
    print('----------')
    print(" Prediction :")
    for i in range(0, len(label)):
        print('     ' + label[i] + ' : ' + "{0:.2f}%".format(prediction[0][i] * 100.))

    print()
    print('RESULTAT : ' + word + ' : ' + "{0:.2f}%".format(pred))
    print('temps prediction : ' + "{0:.2f}secs".format(end-start))

    print('----------')

In [194]:
"""
# On definit les chemins d'acces au différentes hyper parametre
"""

modelPath = '/content/trainedModel/moModel.hdf5'
imagePath =  '/content/Dataset/baklava/1784.jpg'
imageSize = (50,50)
label = ['applepie', 'baby_black_ribs', 'baklava', 'beet_salad']

predict(modelPath, imagePath,imageSize, label)




Chargement du modèle :


Model chargé.

----------
 Prediction :
     applepie : 21.75%
     baby_black_ribs : 13.77%
     baklava : 35.55%
     beet_salad : 6.25%

RESULTAT : baklava : 35.55%
temps prediction : 0.22secs
----------
